In [98]:
import pandas as pd
import numpy as np

# 절대 모멘텀
1. 주식 데이터 로드 
2. 파생변수 년월 파생변수 생성 -> Date 컬럼에서 년-월 추출하여 대입
3. 년, 월 별 마지막날의 month_last_dt 데이터프레임을 생성
4. 전월의 종가를 가지는 파생변수 하나 생성
5. 전년도의 종가를 가지는 파생변수 하나 생성
6. 전월, 전년도의 종가를 가지고 거래내역 생성
7. 수익율 계산

In [99]:
df = pd.read_csv("../csv/GDX.csv", index_col='Date')
df.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2006-05-22,36.520000,37.290001,35.869999,37.230000,34.280224,197100
2006-05-23,37.750000,39.220001,37.750000,37.959999,34.952393,620900
2006-05-24,37.130001,37.570000,35.869999,36.520000,33.626480,638600


In [100]:
# index를 시계열 데이터로 변경
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

In [101]:
from datetime import datetime

In [102]:
# datetime 라이브러리를 이용하여 시계열로 변경
test_df = df.copy()

In [103]:
# case1
test_df = test_df.reset_index()
test_df['Date'] = test_df['Date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
test_df.set_index('Date')

TypeError: strptime() argument 1 must be str, not Timestamp

In [ ]:
# case2
test_df.index

def change(x):
    return datetime.strptime(x, '%Y-%m-%d')

test_df.index = list(map(change, test_df.index))

In [ ]:
test_df.head(2)

In [ ]:
# map 함수
x = [1,2,3,4]
def change(x):
    return x**2

list(map(lambda y : y**2, x))

In [ ]:
df.index

In [ ]:
# 새로운 파생변수 년-월의 데이터를 대입
df['STD-YM'] = list(map(lambda x : datetime.strftime(x, '%Y-%m'), df.index))

In [ ]:
df.head(2)

In [ ]:
(df['STD-YM'] != df.shift(-1)['STD-YM']).iloc[24:30]

In [ ]:
# 월별 마지막 인덱스만 추출하여 새로운 데이터프레임에 대입

# case1 (shift 이용)
# 현재의 STD-YM의 값과 다음 행의 STD-YM의 값이 다른 경우 
df.loc[df['STD-YM'] != df.shift(-1)['STD-YM']]

In [ ]:
last_df = df.loc[df['STD-YM'] == "2006-06"]
last_df.tail(1)

In [ ]:
# case2 (for문)
month_list_df = pd.DataFrame()

_list = df['STD-YM'].unique()

for i in _list:
    last_df = df.loc[df['STD-YM'] == i].tail(1)
    month_list_df = pd.concat([month_list_df, last_df])

month_list_df

In [ ]:
x = pd.Series([1,1,1,2,2,4,4,4,4,7,7,7,7,7,7])

x.unique()

In [ ]:
# 전월의 종가(Adj Close)를 가지는 파생변수(BF_1M)을 생성하고 결측치는 0으로 대체
# 전년도의 종가(Adj Close)를 가지는 파생변수(BF_12M)을 생성하고 결측치는 0으로 대체

month_list_df['BF_1M'] = month_list_df.shift(1)['Adj Close'].fillna(0)

In [ ]:
month_list_df.head(3)

In [ ]:
month_list_df['BF_12M'] = month_list_df.shift(12)['Adj Close'].fillna(0)

In [ ]:
month_list_df.iloc[[0,12], [4, 8]]

,Adj Close,BF_12M
Date,,
2006-05-31,35.771877,0.000000
2007-05-31,36.162109,35.771877


In [ ]:
# 거래내역 추가 
# 구매 조건 -> ((전월 종가 / 전 년도 종가) - 1)이 값이 0보다 크고 무한대가 아닌 경우

df['trade'] = ""

for i in month_list_df.index:
    signal = ""

    # 절대 모멘텀 계산
    momentum_index = month_list_df.loc[i,'BF_1M'] / \
        month_list_df.loc[i,'BF_12M'] - 1

    # 절대 모멘텀 지표에 따라서 True 와 False 로 구분
    flag = True if((momentum_index > 0) and (momentum_index != -np.inf ) \
        and (momentum_index != np.inf)) else False
    
    if flag:
        signal = 'buy'
    
    print('날짜 :', i, "모멘텀 인덱스 :", momentum_index, "flag :", flag, \
        "signal :", signal)
    
    df.loc[i, 'trade'] = signal

날짜 : 2006-05-31 00:00:00 모멘텀 인덱스 : nan flag : False signal : 
날짜 : 2006-06-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-07-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-08-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-09-29 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-10-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-11-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2006-12-29 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2007-01-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2007-02-28 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2007-03-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2007-04-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 2007-05-31 00:00:00 모멘텀 인덱스 : 0.02123525695897932 flag : True signal : buy
날짜 : 2007-06-29 00:00:00 모멘텀 인덱스 : 0.014827231670955676 flag : True signal : buy
날짜 : 2007-07-31 00:00:00 모멘텀 인덱스 : -0.019604388455949806 flag : False signal : 
날짜 : 2007-08-31

C:\Users\moons\AppData\Local\Temp\ipykernel_2528\2280660222.py:10: RuntimeWarning: invalid value encountered in scalar divide
  momentum_index = month_list_df.loc[i,'BF_1M'] / \
C:\Users\moons\AppData\Local\Temp\ipykernel_2528\2280660222.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  momentum_index = month_list_df.loc[i,'BF_1M'] / \


In [ ]:
df['trade'].value_counts()

       3234
buy      61
Name: trade, dtype: int64

### 수익율 계산
1. 파생변수(return) 값은 1로 생성
2. rtn = 1.0, buy, sell 변수는 0으로 생성
3. 반복문을 이용해서 현재의 trade가 buy이고 전 행의 trade가 ""는 경우 구매 날
4. 전 행의 trade가 buy이고 현재의 trade가 ""인 경우 판매 날
5. 수익율 계산해서 return 대입

### 누적 수익율 계산
1. acc_rtn = 1.0 생성
2. return의 항목들을 누적 곱하여 acc_rtn 컬럼에 대입
3. 최종적으로 acc_rtn을 출력

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3295 entries, 2006-05-22 to 2019-06-24
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3295 non-null   float64
 1   High       3295 non-null   float64
 2   Low        3295 non-null   float64
 3   Close      3295 non-null   float64
 4   Adj Close  3295 non-null   float64
 5   Volume     3295 non-null   int64  
 6   STD-YM     3295 non-null   object 
 7   trade      3295 non-null   object 
dtypes: float64(5), int64(1), object(2)
memory usage: 360.7+ KB


In [ ]:
len(df.index.unique())

3295

In [ ]:
df.index.value_counts().sort_values(ascending=False)

2006-05-22    1
2006-06-09    1
2006-05-24    1
2006-05-25    1
2006-05-26    1
             ..
2019-06-17    1
2019-06-18    1
2019-06-19    1
2019-06-20    1
2019-06-24    1
Name: Date, Length: 3295, dtype: int64

In [ ]:
df['return'] = 1
rtn = 1.0
buy = 0
sell = 0

for i in df.index:
    # 구매한 날을 체크 , 구매가 지정
    if (df.loc[i, 'trade'] == "buy") and (df.shift(1).loc[i, 'trade'] == ""):
        buy = df.loc[i, 'Adj Close']
        print('진입일 :', i, '구입 가격 :', buy)
    # 판매한 날을 체크, 판매가 지정, 수익율 계산
    elif (df.loc[i, 'trade'] == "") and (df.shift(1).loc[i, 'trade'] == 'buy'):
        sell = df.loc[i, 'Adj Close']
        rtn = (sell - buy) / buy + 1
        df.loc[i, 'return'] = rtn
        print('판매일 :', i, "판매 가격 :", sell, "수익율 :", rtn)



In [ ]:
acc_rtn = 1.0

for i in df.index:
    acc_rtn *= df.loc[i, 'return']
    df.loc[i, 'acc_rtn'] = acc_rtn

print('누적 수익율 :', acc_rtn)

누적 수익율 : 1.165001849964831


In [ ]:
# 누적곱를 해주는 함수 
df['return'].cumprod()

Date
2006-05-22    1.000000
2006-05-23    1.000000
2006-05-24    1.000000
2006-05-25    1.000000
2006-05-26    1.000000
                ...   
2019-06-18    1.165002
2019-06-19    1.165002
2019-06-20    1.165002
2019-06-21    1.165002
2019-06-24    1.165002
Name: return, Length: 3295, dtype: float64

# 절대 모멘텀을 함수화
1. 첫 번째 함수
    - 매개변수 1개(데이터프레임)
    - 데이터프레임에 결측치와 이상치를 제거 
    - 수정종가를 제외한 나머지 컬럼은 삭제
    - 인덱스를 시계열로 변경
    - 'STD-YM' 파생변수를 생성하여 인덱스의 년-월 추출해서 대입
2. 두 번째 함수
    - 매개변수 1개(데이터프레임)
    - 새로운 데이터프레임(df2) 생성
    - 인자값으로 받아온 데이터프레임에서 년-월별 마지막 데이터들을 새로운 데이터프레임에 대입
    - df2에 파생변수 2개 생성
        - 'BF_1M' : 전월의 종가, 결측치는 0으로 대체
        - 'BF_12M' : 전 년도의 종가, 결측치는 0으로 대체
    - df2를 리턴
3. 세 번째 함수
    - 매개변수 2개(첫번째 함수의 결과(df1), 두 번째 함수의 결과(df2))
    - df1에 trade 컬럼을 생성, 값은  ""
    - df1에 return 컬럼을 생성, 값은 1
    - df2의 값들을 이용하여 momentum_index를 구하고 df1에 거래 내역 삽입
    - df1의 거래 내역을 이용하여 수익율 return 컬럼에 대입
    - return 컬럼의 데이터를 가지고 누적수익율(acc_rtn)에 대입
    - 누적 수익율을 print를 이용하여 출력
    - df1을 리턴

In [104]:
# 1번 함수 생성
def first(df):
    # 인덱스가 Date가 아니면? Date컬럼을 인덱스로 변경
    if 'Date' in df.columns:
        df = df.set_index('Date')
    # 결측치와 이상치를 제거
    df = df.loc[~df.isin([np.nan, np.inf, -np.inf]).any(axis='columns')]
    # 수정종가를 제외한 나머지 컬럼 삭제
    df = df[['Adj Close']]
    # 인덱스를 시계열로 변경
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
    # STD-YM 컬럼을 생성 
    df['STD-YM'] = list(map(lambda x : datetime.strftime(x, '%Y-%m'), df.index))

    return df

In [105]:
data = pd.read_csv("../csv/GDX.csv")

In [106]:
df1 = first(data)

In [107]:
# 2번함수 
def second(df):
    # df2 = pd.DataFrame()

    # _list = df['STD-YM'].unique()

    # for i in _list:
    #     last_df = df.loc[df['STD-YM'] == i].tail(1)
    #     df2 = pd.concat([df2, last_df])

    df2 = df.loc[df['STD-YM'] != df.shift(-1)['STD-YM']]
    
    df2['BF_1M'] = df2.shift(1)['Adj Close'].fillna(0)
    df2['BF_12M'] = df2.shift(12)['Adj Close'].fillna(0)

    return df2

In [108]:
df2 = second(df1)

C:\Users\moons\AppData\Local\Temp\ipykernel_2528\449586186.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['BF_1M'] = df2.shift(1)['Adj Close'].fillna(0)
C:\Users\moons\AppData\Local\Temp\ipykernel_2528\449586186.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['BF_12M'] = df2.shift(12)['Adj Close'].fillna(0)


In [109]:
# 3번 함수
def third(df1, df2):
    df1['trade'] = ""
    df1['return'] = 1

    for i in df2.index:
        signal = ""

        momentum_index = df2.loc[i, 'BF_1M'] / df2.loc[i, 'BF_12M'] - 1

        flag = True if((momentum_index > 0) \
                and (momentum_index != np.inf) \
                and (momentum_index != -np.inf)) else False
        
        if flag : 
            signal = 'buy'
        
        df1.loc[i, 'trade'] = signal
    
    rtn = 1.0
    buy = 0
    sell = 0

    for i in df1.index:
        if (df1.loc[i, 'trade'] == "buy") and (df1.shift(1).loc[i, 'trade'] == ""):
            buy = df1.loc[i, 'Adj Close']
            print('구입일 :', i, "구입 가격 :", buy)
        elif (df1.loc[i, 'trade'] == "") and (df1.shift(1).loc[i, 'trade'] == "buy"):
            sell = df1.loc[i, 'Adj Close']
            print('판매일 :', i, "판매 가격 :", sell)
            rtn = (sell - buy) / buy + 1
            df1.loc[i, 'return'] = rtn
    

    acc_rtn = 1.0

    for i in df1.index:
        acc_rtn *= df1.loc[i, 'return']
        df1.loc[i, 'acc_rtn'] = acc_rtn
    print(acc_rtn)

    return df1


In [110]:
third(df1, df2)

C:\Users\moons\AppData\Local\Temp\ipykernel_2528\2487612304.py:9: RuntimeWarning: invalid value encountered in scalar divide
  momentum_index = df2.loc[i, 'BF_1M'] / df2.loc[i, 'BF_12M'] - 1
C:\Users\moons\AppData\Local\Temp\ipykernel_2528\2487612304.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  momentum_index = df2.loc[i, 'BF_1M'] / df2.loc[i, 'BF_12M'] - 1


구입일 : 2007-05-31 00:00:00 구입 가격 : 36.162109
판매일 : 2007-06-01 00:00:00 판매 가격 : 36.993214
구입일 : 2007-06-29 00:00:00 구입 가격 : 34.989338
판매일 : 2007-07-02 00:00:00 판매 가격 : 35.866615
구입일 : 2007-09-28 00:00:00 구입 가격 : 41.878242
판매일 : 2007-10-01 00:00:00 판매 가격 : 42.312256
구입일 : 2007-10-31 00:00:00 구입 가격 : 46.72633
판매일 : 2007-11-01 00:00:00 판매 가격 : 45.082588
구입일 : 2007-11-30 00:00:00 구입 가격 : 42.810921
판매일 : 2007-12-03 00:00:00 판매 가격 : 43.235706
구입일 : 2007-12-31 00:00:00 구입 가격 : 43.033947
판매일 : 2008-01-02 00:00:00 판매 가격 : 46.235912
구입일 : 2008-01-31 00:00:00 구입 가격 : 47.278187
판매일 : 2008-02-01 00:00:00 판매 가격 : 46.151402
구입일 : 2008-02-29 00:00:00 구입 가격 : 49.879185
판매일 : 2008-03-03 00:00:00 판매 가격 : 51.287678
구입일 : 2008-03-31 00:00:00 구입 가격 : 44.78986
판매일 : 2008-04-01 00:00:00 판매 가격 : 43.559784
구입일 : 2008-04-30 00:00:00 구입 가격 : 40.939995
판매일 : 2008-05-01 00:00:00 판매 가격 : 40.047958
구입일 : 2008-05-30 00:00:00 구입 가격 : 43.212353
판매일 : 2008-06-02 00:00:00 판매 가격 : 43.57856
구입일 : 2008-06-30 00:00:00 구입 가격 : 4

,Adj Close,STD-YM,trade,return,acc_rtn
Date,,,,,
2006-05-22,34.280224,2006-05,,1.0,1.000000
2006-05-23,34.952393,2006-05,,1.0,1.000000
2006-05-24,33.626480,2006-05,,1.0,1.000000
2006-05-25,35.283867,2006-05,,1.0,1.000000
2006-05-26,35.495640,2006-05,,1.0,1.000000
...,...,...,...,...,...
2019-06-18,23.670000,2019-06,,1.0,1.165002
2019-06-19,24.000000,2019-06,,1.0,1.165002
2019-06-20,25.049999,2019-06,,1.0,1.165002
